In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings

import matplotlib.pyplot as plt
from pandas.plotting import table

import holoviews as hv
import os
import holoviews as hv
# from selenium import webdriver
# from selenium.webdriver.firefox.service import Service


# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the data into a Pandas DataFrame
maryland_montgomoryCounty_data = pd.read_csv(
    "Resources/Real_Property_Tax_-_2023.csv",
    index_col=0)

# Display sample data
maryland_montgomoryCounty_data.head(10)

,RESIDENCE,PROPERTY ADDRESS,CITY,STATE,ZIP CODE,TAX CLASS,ASSESSMENT,BARNESVILLE PROPERTY TAX,BATTERY PARK PROPERTY TAX,BAY RESTORATION FUND,...,MUNICIPAL HOMESTEAD CREDIT,MUNICIPAL SWIMCLUB,OTHER PARTIAL EXEMPTION,PUBLIC SAFETY OFFICERS TAX CR,STATE HOMEOWNER'S CREDIT,STATE HOMESTEAD CREDIT,SWIMCLUB,TOTAL CREDITS,BILL TOTAL,GEO
PARCEL CODE,,,,,,,,,,,,,,,,,,,,,
11958,NOT A PRINCIPAL RESIDENCE,1ST ST,NaN,MD,NaN,R058,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,189.0,POINT (-78.921683651 39.655141985)
18117,NOT A PRINCIPAL RESIDENCE,CLOPPER RD,NaN,MD,NaN,R042,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-77.642166643 39.676171137)
21365,NOT A PRINCIPAL RESIDENCE,FREDERICK RD,NaN,MD,NaN,R042,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-77.705167887 39.606728712)
25567,NOT A PRINCIPAL RESIDENCE,CLARKSBURG RD,NaN,MD,NaN,R042,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-77.222430895 39.330228088)
50813,NOT A PRINCIPAL RESIDENCE,NaN,NaN,MD,NaN,R053,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-76.15872999999993 38.81819928400006)
235130,NOT A PRINCIPAL RESIDENCE,NELSON ST,NaN,MD,NaN,R050,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-77.412159628 39.443920035)
255024,NOT A PRINCIPAL RESIDENCE,GOOD HOPE RD,NaN,MD,NaN,R042,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.416461348 39.679337216)
271068,NOT A PRINCIPAL RESIDENCE,COLUMBIA PIK,NaN,MD,NaN,R042,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-76.86709 39.21106)
399025,NOT A PRINCIPAL RESIDENCE,RIVER RD,NaN,MD,NaN,R042,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.041273411 39.482321624)


In [3]:
# Create a copy of the df_market_data DataFrame
df_maryland_montgomoryCounty_data = maryland_montgomoryCounty_data.copy()

In [4]:
# Generate summary statistics
df_maryland_montgomoryCounty_data.describe()

,ZIP CODE,ASSESSMENT,BARNESVILLE PROPERTY TAX,BATTERY PARK PROPERTY TAX,BAY RESTORATION FUND,BROOKEVILLE PROPERTY TAX,CH CH SECTION 3 PROPERTY TAX,CH CH SECTION 5 PROPERTY TAX,CH CH TOWN PROPERTY TAX,CH CH VIEW PROPERTY TAX,...,MUNICIPAL HOMEOWNERS' CREDIT,MUNICIPAL HOMESTEAD CREDIT,MUNICIPAL SWIMCLUB,OTHER PARTIAL EXEMPTION,PUBLIC SAFETY OFFICERS TAX CR,STATE HOMEOWNER'S CREDIT,STATE HOMESTEAD CREDIT,SWIMCLUB,TOTAL CREDITS,BILL TOTAL
count,326022.000000,3.451900e+05,345190.000000,345190.000000,345190.000000,345190.000000,345190.000000,345190.0,345190.000000,345190.000000,...,345190.000000,345190.000000,345190.000000,3.451900e+05,345190.000000,345190.000000,345190.000000,345190.000000,3.451900e+05,3.451900e+05
mean,20868.621979,6.376942e+05,0.042097,0.327757,2.117269,0.118784,0.209588,0.0,0.382922,0.211106,...,-0.547519,-0.426024,-0.022916,-2.910458e+01,-4.912821,-11.391284,-1.020838,-2.527376,-4.667258e+02,7.720315e+03
std,58.024525,3.151632e+06,3.349276,16.523189,11.070395,10.151387,8.008434,0.0,8.291646,7.414833,...,19.013994,13.684993,11.885238,4.529333e+03,109.729173,165.099912,11.824174,250.713216,4.566560e+03,3.797671e+04
min,20166.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,-1292.600000,-2144.250000,-6911.060000,-1.631135e+06,-2500.000000,-5054.000000,-1587.510000,-58695.020000,-1.631135e+06,0.000000e+00
25%,20851.000000,2.786000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-6.920000e+02,3.238670e+03
50%,20874.000000,4.302000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-6.920000e+02,5.221625e+03
75%,20901.000000,6.560670e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,7.983918e+03
max,37215.000000,6.109545e+08,694.710000,6062.970000,60.000000,1455.750000,1037.050000,0.0,2372.200000,503.580000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,7.162195e+06


In [9]:
# Load the state property tax data into a Pandas DataFrame
state_property_taxes_data = pd.read_csv(
    "Resources/Property_Taxes_by_State_County_Median_Property_Tax_Bills.csv",
    index_col=0)

# Display sample data
state_property_taxes_data.head(5)

,State,"Median Property Taxes Paid by County, 2021 (5-year estimate)"
County,,
Autauga County,Alabama,491
Baldwin County,Alabama,718
Barbour County,Alabama,338
Bibb County,Alabama,263
Blount County,Alabama,416


In [10]:
state_property_taxes_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3221 entries, Autauga County to Yauco Municipio
Data columns (total 2 columns):
 #   Column                                                        Non-Null Count  Dtype 
---  ------                                                        --------------  ----- 
 0   State                                                         3221 non-null   object
 1   Median Property Taxes Paid by County, 2021 (5-year estimate)  3203 non-null   object
dtypes: object(2)
memory usage: 75.5+ KB


In [32]:
# Reset index
state_property_taxes_df = state_property_taxes_data.copy()
state_property_taxes_df = state_property_taxes_df.reset_index()
state_property_taxes_df.tail()

,County,State,"Median Property Taxes Paid by County, 2021 (5-year estimate)"
3216,Vega Baja Municipio,Puerto Rico,376
3217,Vieques Municipio,Puerto Rico,NaN
3218,Villalba Municipio,Puerto Rico,473
3219,Yabucoa Municipio,Puerto Rico,200
3220,Yauco Municipio,Puerto Rico,NaN


In [40]:
# Drop all null values
state_property_taxes_cleaned_df = state_property_taxes_df.dropna()

In [41]:
state_property_taxes_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3203 entries, 0 to 3219
Data columns (total 3 columns):
 #   Column                                                        Non-Null Count  Dtype 
---  ------                                                        --------------  ----- 
 0   County                                                        3203 non-null   object
 1   State                                                         3203 non-null   object
 2   Median Property Taxes Paid by County, 2021 (5-year estimate)  3203 non-null   object
dtypes: object(3)
memory usage: 100.1+ KB


In [62]:
# Remove commas from the 'Median Property Taxes Paid by County, 2021 (5-year estimate)' column
state_property_taxes_cleaned_df['Median Property Taxes Paid by County, 2021 (5-year estimate)'] = \
state_property_taxes_cleaned_df['Median Property Taxes Paid by County, 2021 (5-year estimate)'].str.replace(',', '')

In [64]:
# Change 'Median Property Taxes Paid by County, 2021 (5-year estimate)' column to integer data type
state_property_taxes_cleaned_df['Median Property Taxes Paid by County, 2021 (5-year estimate)'] = \
state_property_taxes_cleaned_df['Median Property Taxes Paid by County, 2021 (5-year estimate)'].astype(int)

In [65]:
state_property_taxes_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3203 entries, 0 to 3219
Data columns (total 3 columns):
 #   Column                                                        Non-Null Count  Dtype 
---  ------                                                        --------------  ----- 
 0   County                                                        3203 non-null   object
 1   State                                                         3203 non-null   object
 2   Median Property Taxes Paid by County, 2021 (5-year estimate)  3203 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 100.1+ KB


In [66]:
# Review dataframe for duplicates
duplicates = state_property_taxes_cleaned_df[state_property_taxes_cleaned_df['County'].duplicated()]
duplicates

,County,State,"Median Property Taxes Paid by County, 2021 (5-year estimate)"
118,Calhoun County,Arkansas,431
122,Clay County,Arkansas,399
123,Cleburne County,Arkansas,731
131,Dallas County,Arkansas,466
135,Franklin County,Arkansas,570
...,...,...,...
3131,Lincoln County,Wyoming,1395
3134,Park County,Wyoming,1782
3135,Platte County,Wyoming,1244
3136,Sheridan County,Wyoming,1698


In [67]:
clay_county = state_property_taxes_cleaned_df[state_property_taxes_cleaned_df['County'] == "Clay County"]
clay_county

,County,State,"Median Property Taxes Paid by County, 2021 (5-year estimate)"
13,Clay County,Alabama,307
122,Clay County,Arkansas,399
330,Clay County,Florida,1732
417,Clay County,Georgia,649
608,Clay County,Illinois,1573
708,Clay County,Indiana,536
810,Clay County,Iowa,1686
902,Clay County,Kansas,1773
1019,Clay County,Kentucky,531
1328,Clay County,Minnesota,2454


In [84]:
# Calculate property taxes by state
total_property_taxes_by_state = state_property_taxes_cleaned_df.groupby('State')\
                                ['Median Property Taxes Paid by County, 2021 (5-year estimate)'].sum()

# Sort the results in descending order
total_property_taxes_by_state_sorted = total_property_taxes_by_state.sort_values(ascending=False)

total_property_taxes_by_state_sorted

State
Texas                   474073
New York                271329
Illinois                259617
Virginia                234642
Georgia                 210024
Wisconsin               205783
California              202439
Iowa                    189955
Kansas                  185309
New Jersey              170273
Ohio                    167299
Pennsylvania            164816
Minnesota               162502
Nebraska                161243
Michigan                156098
North Carolina          126902
Missouri                123718
Kentucky                115579
Florida                 106944
South Dakota            103360
Washington              101660
Indiana                  94848
Montana                  88541
Oregon                   87619
Tennessee                86494
Colorado                 84015
Maryland                 73214
North Dakota             68722
Mississippi              67845
Massachusetts            64735
Oklahoma                 64479
Vermont                  59947
Id

In [88]:
# View Alabama property taxes 
alabama_values = state_property_taxes_cleaned_df[state_property_taxes_cleaned_df['State'] == "Alabama"]
alabama_values

# Sort the results in descending order
alabama_taxes_sorted = alabama_values.sort_values(by='Median Property Taxes Paid by County, 2021 (5-year estimate)', ascending=False)
alabama_taxes_sorted.reset_index(drop=True)

,County,State,"Median Property Taxes Paid by County, 2021 (5-year estimate)"
0,Shelby County,Alabama,1167
1,Jefferson County,Alabama,1149
2,Lee County,Alabama,972
3,Madison County,Alabama,905
4,Mobile County,Alabama,781
...,...,...,...
62,Bibb County,Alabama,263
63,Coosa County,Alabama,259
64,Sumter County,Alabama,252
65,Lamar County,Alabama,234


---

### Prepare the Data

---

### Find the Best Value for k Using the Original Data.

---

### Cluster with K-means Using the Original Data

---

### Optimize Clusters with Principal Component Analysis.

---

### Find the Best Value for k Using the PCA Data

---

### Cluster with K-means Using the PCA Data

---

### Visualize and Compare the Results


---